In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import os
import re

from tqdm import tqdm_notebook
from bs4 import BeautifulSoup
from utils import *
from row_utils import *
from string_utils import *

# Set directory & name of save file
results = []
save_file_name = "check1_df"

path_prefix = "./Check_20190717/"
all_2004 = os.listdir(path_prefix)
all_2004 = [x for x in all_2004 if ('DS_Store' not in x)]
all_2004 = [x for x in all_2004 if ('xlsx' not in x)]

# Recording which files have been scraped 
cik_done_dict = get_cik_done(save_file_name)
#cik_done_dict = {}

In [109]:
# Scraping the remaining files
results = []
for i in tqdm_notebook(range(len(all_2004))):
    
    fname = all_2004[i]
    test_f = open(path_prefix + fname)
    wholetext = test_f.read()
    cikbase = re.search(r"CENTRAL INDEX KEY:\s*?\d{10}\b", wholetext, re.I).group()
    cik_re = re.split(r'\s', cikbase)[-1]        

    datadatebase = re.search(r"CONFORMED PERIOD OF REPORT:\s*?\d{8}\b", wholetext, re.I).group() 
    datadate_re = re.split(r'\s', datadatebase)[-1] 
    if int(cik_re) in cik_done_dict:
        if int(datadate_re) in cik_done_dict[int(cik_re)]:
            print("Already processed this file")
            continue
    
    wholetext = wholetext.lower()
    tbl_flag = 0 # 1 if "contractual obligations" table exists
    row_flag = 0 # 1 if "purchasing obligations" row exists
    multiplier = 1 # 1 by default
    
    table = scrape_table(wholetext)
    rows = []
    if (not table) or ('-----' in table.text):
        multiplier, lines, tbl_list = scrape_text_table(wholetext)
        if tbl_list != None:
            tbl_flag = 1
            rows = scrape_text_rows(lines, tbl_list)
    else:
        tbl_flag = 1
        rows = scrape_rows(table)
        multiplier = get_multiplier_from_tbl(table)
        
    for row in rows:
        row_flag = 1
        
        continue_flag = 0
        all_entries = [float(row['<1']), float(row['1-3']), float(row['3-5']), float(row['>5'])]
        if np.sum(all_entries) == 0:
            continue_flag = 1
        if continue_flag == 1:
            continue
            
        result = {'fname': fname, 'cik': cik_re, 'datadate': datadate_re, 'tbl': tbl_flag, 
                  'row': row_flag, '<1': row['<1'], '1-3':row['1-3'], 
                  '3-5':row['3-5'], '>5':row['>5'], 
                  'category': clean_category_name(row['category']), 
                  'multiplier': multiplier}
        if 'total' in row.keys():
                result['total'] = row['total']
        results.append(result)

    # No table or no row
    if (tbl_flag == 0) or (row_flag == 0):
        result = {'fname': fname, 'cik': cik_re, 'datadate': datadate_re, 'tbl': tbl_flag, 'row': row_flag, 
                  'total': '-', '<1': '-', '1-3':'-', '3-5':'-', '>5':'-', 'multiplier': multiplier, 'category': 'None'}

        results.append(result)
    pd.DataFrame(results).to_csv(save_file_name)
    print("File: ", i, "\tTable: ", tbl_flag, "Row: ", row_flag)


/Users/divyas/Documents/real_docs/G1/sec/utils.py:33: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 33 of the file /Users/divyas/Documents/real_docs/G1/sec/utils.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  parsed_html = BeautifulSoup(wholetext)


File:  0 	Table:  1 Row:  1
File:  1 	Table:  1 Row:  1
File:  2 	Table:  1 Row:  1
File:  3 	Table:  1 Row:  1
[]
File:  4 	Table:  1 Row:  0
[]
File:  5 	Table:  1 Row:  0
File:  6 	Table:  1 Row:  1
File:  7 	Table:  1 Row:  1
File:  8 	Table:  1 Row:  1
[]
File:  9 	Table:  1 Row:  0
File:  10 	Table:  1 Row:  0
File:  11 	Table:  1 Row:  1
File:  12 	Table:  1 Row:  1
File:  13 	Table:  1 Row:  1
File:  14 	Table:  1 Row:  1
File:  15 	Table:  1 Row:  1
File:  16 	Table:  1 Row:  1
File:  17 	Table:  1 Row:  0
File:  18 	Table:  1 Row:  0
[]
File:  19 	Table:  1 Row:  0
File:  20 	Table:  1 Row:  1
File:  21 	Table:  1 Row:  0
File:  22 	Table:  1 Row:  1
[['purchase obligations', '9333', '0', '0', '0', '9333']]
['purchase obligations', '9333', '0', '0', '0', '9333'] []
File:  23 	Table:  1 Row:  1
File:  24 	Table:  1 Row:  1
File:  25 	Table:  1 Row:  1
[['employment contracts', '4253000', '2162000', '2091000', '0', '0']]
['employment contracts', '4253000', '2162000', '2091000',

In [110]:
#results_df = pd.DataFrame(results)
results_df = pd.read_csv(save_file_name)
row_check = results_df[(results_df['row'] == 0) & (results_df['tbl'] == 1)] # table but no row
none_check = results_df[(results_df['row'] == 0) & (results_df['tbl'] == 0)] # no table no row
last_check = results_df[(results_df['row'] == 1) & (results_df['tbl'] == 1)] # table and row, other issue?
print("row & table found: ", len(set(last_check['fname'])))
print("table found: ", len(set(row_check['fname'])))
print("nothing found: ", len(set(none_check['fname'])))

row & table found:  23
table found:  10
nothing found:  0


In [111]:
row_check

,Unnamed: 0,1-3,3-5,<1,>5,category,cik,datadate,fname,multiplier,row,tbl,total
8,8,-,-,-,-,None,818677,20090331,0000939057-09-000175.txt,1000,0,1,-
9,9,-,-,-,-,None,1046050,20090930,0000939057-09-000370.txt,1000,0,1,-
18,18,-,-,-,-,None,313337,20081231,0000313337-09-000011.txt,1000,0,1,-
19,19,-,-,-,-,None,1443818,20081231,0001445866-09-000016.txt,1,0,1,-
26,26,-,-,-,-,None,1116734,20090331,0001144204-09-037210.txt,1,0,1,-
27,27,-,-,-,-,None,896400,20090930,0001437749-09-002044.txt,1000,0,1,-
28,28,-,-,-,-,None,796655,20081231,0001157523-09-003268.txt,1,0,1,-
31,31,-,-,-,-,None,1010470,20090630,0000939057-09-000247.txt,1,0,1,-
44,44,-,-,-,-,None,355019,20090630,0000355019-09-000009.txt,1,0,1,-
46,46,-,-,-,-,None,727920,20081231,0000089024-09-000140.txt,1000000,0,1,-


# Examining one file

In [133]:
test_f = open(path_prefix + "0000939057-09-000247.txt")
wholetext = test_f.read()
cik_re = get_cik(wholetext)
datadate_re = get_datadate(wholetext)  
print("CIK: ", cik_re, "Date: ", datadate_re)
wholetext = wholetext.lower()

# Text table debugging
#text_chunk, multiplier, lines, tbl_list = scrape_text_table(wholetext)
#mult, lines, tbl_list = scrape_text_table(wholetext)
#rows = scrape_text_rows(lines, tbl_list)
# HTML debugging 
scores, tables = scrape_table(wholetext)
#table = scrape_table(wholetext)
#rows = scrape_rows(table)
#mult = get_multiplier_from_tbl(table)
rows, mult

CIK:  0001010470 Date:  20090630


([], 1)

In [147]:
#print(scores)
table_to_list(tables[np.where(scores == np.max(scores)-2)[0][10]])

IndexError: index 10 is out of bounds for axis 0 with size 10

In [13]:
get_table_headers_from_list(lines)

['obligations', '1', 'year', 'years', 'years', '5', 'years', 'total']

In [74]:
tbl_list

[['contractual obligations and commercial commitments. at december',
  '31,',
  '2008,',
  'our'],
 ['after', '2009', '2010', '2011', '2012', '2013', '2013', 'total'],
 ['operating leases', '606', '607', '474', '55', '14', '--', '1,756'],
 ['purchase', 'obligations', '4,852', '--', '--', '--', '--', '--', '4,852'],
 ['-------',
  '------',
  '------',
  '------',
  '------',
  '------',
  '------- total',
  '$6,075',
  '$1,223',
  '$1,090',
  '671',
  '630',
  '$3,489',
  '$13,178'],
 ['',
  'our purchase obligations are amounts committed under legally enforceable'],
 ['short duration of the securities in which we invest and the conservativenature of our investment portfolio, a',
  '10%',
  'move in interest rates over a'],
 ['one-year period would have an immaterial effect of approximately',
  '$20,000',
  'of'],
 ['decreased interest income on our financial position, results of operationsand cash flows. if the yen strengthens against the dollar by',
  '10%,',
  'the'],
 ['have an imm

In [75]:
text_chunk
l = text_chunk.split('\n')
numbers_row_indices = [hasNumbers(x) for x in l]
start_row_idx = np.where(np.array(numbers_row_indices) == True)[0][0]
end_row_idx = np.where(np.array(numbers_row_indices) == True)[0][-1]

In [52]:
l[start_row_idx:end_row_idx+1]

['                   payments due by period at december 31, 2008',
 '                                 (in thousands)',
 '',
 '                                            less than     one to       three to     more than',
 'contractual obligations           total      one year   three years   five years   five years',
 '- -----------------------           -----     ---------   -----------   ----------   ----------',
 '<s>                             <c>         <c>         <c>           <c>          <c>',
 'certificates of deposit and     $ 143,190   $ 110,433   $    25,548   $    5,574   $    1,635',
 '  other time deposits',
 'short-term debt',
 '  obligations                       3,911       3,911             -            -            -',
 'minimum operating lease',
 '  obligations                       1,745         817           849           63           16',
 '                                ---------   ---------   -----------   ----------   ----------',
 '       total         

In [56]:
lines = l[start_row_idx:end_row_idx+1]
line_no = 0
condensed_lines = []
while line_no < len(lines):
    if hasNumbers(lines[line_no]):
        if line_no > 1 and not hasNumbers(lines[line_no-1]):
            condensed_lines.append(lines[line_no-1] + lines[line_no])
        else:
            condensed_lines.append(lines[line_no])
    line_no += 1
condensed_lines

['                   payments due by period at december 31, 2008',
 '<s>                             <c>         <c>         <c>           <c>          <c>certificates of deposit and     $ 143,190   $ 110,433   $    25,548   $    5,574   $    1,635',
 'short-term debt  obligations                       3,911       3,911             -            -            -',
 'minimum operating lease  obligations                       1,745         817           849           63           16',
 '                                ---------   ---------   -----------   ----------   ----------       total                    $ 148,846   $ 115,161   $    26,397   $    5,637   $    1,651',
 "sensitivity  models and monitor  earnings at risk. these limits are based on thebank's  exposure to a 100 bp and 200 bp immediate  and  sustained  parallel rate",
 '                                       24']

In [34]:
lines = condensed_lines
keywords = HDR_KWS + ROW_KWS
lines = [x for x in lines if (hasKeywords(x, keywords))]
lines = [x.split(' ') for x in lines]
for i, line in enumerate(lines):
    lines[i] = list(filter(lambda a: (a != '') and (a != '$') 
                           and (a !='<u>') and (a !='</u>'), line))    

In [67]:
tbl_list = table_to_list(table)
tbl_list

[['long-term\n      debt', '200.0', '-', '-', '-', '200.0'],
 ['loans\n      from\n      affiliates', '1,325.0', '-', '-', '-', '1,325.0'],
 ['operating\n      leases', '3,356.9', '203.7', '394.0', '403.1', '2,356.1'],
 ['employee\n      benefits', '37.0', '2.5', '6.4', '7.3', '20.8'],
 ['total\n      contractual obligations $',
  '91,511.1',
  '$',
  '2,643.7',
  '$',
  '4,888.0',
  '$',
  '4,954.1',
  '$',
  '79,025.3']]

In [74]:
n_prev_siblings = 6
preceding_text = ' '.join([str(x) for x in itertools.islice(table.previous_siblings, n_prev_siblings)])

In [7]:
node = table
el_list = []
for i in range(20):
    el_list.append(node.parent)
    node = node.parent

In [38]:
prev_text = table.findAllPrevious(text=True)[:500]
prev_text = [x.replace('\xa0', '') for x in prev_text]
prev_text = [x.replace('\n', '') for x in prev_text]
prev_text = [x.replace('$', '') for x in prev_text]
prev_text = [x.replace(' ', '') for x in prev_text]
prev_text = [x for x in prev_text if ('' != x)][:30]
prev_text = ' '.join(prev_text)

In [39]:
get_multiplier_from_string(prev_text)

1000000